In [7]:
import hax
hax.init(experiment='XENON1T',
         pax_version_policy='loose',
         force_reload=False,
         raw_data_access_mode = 'local',
        raw_data_local_path = ['/dali/lgrandi/ctherreau/sim/Fax_HE_normal_pax/raw/'], 
         minitree_paths=[
             '/dali/lgrandi/ctherreau/sim/Fax_HE_normal_pax/minitree/', # 
            ])


In [1]:
import os, sys, time
import numpy as np
from numpy import sqrt, exp, pi, square
import pandas as pd
pd.options.mode.chained_assignment = None        # default='warn'
import matplotlib
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'   # enable if you have a retina display
from scipy.optimize import curve_fit, minimize
from scipy.interpolate import interp1d, UnivariateSpline
import warnings
warnings.filterwarnings('ignore')
from multihist import Histdd, Hist1d
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool
from contextlib import contextmanager

def plt_config(title=None, xlim=None, ylim=None, xlabel=None, ylabel=None, colorbar=False, sci=False):
    for field in ['title', 'xlim', 'ylim', 'xlabel', 'ylabel']:
        if eval(field) != None: getattr(plt, field)(eval(field))
    if isinstance(sci, str): plt.ticklabel_format(style='sci', axis=sci, scilimits=(0,0))
    if isinstance(colorbar,str): plt.colorbar(label=colorbar)
    elif colorbar: plt.colorbar(label = '$Number\ of\ Entries$')

@contextmanager
def initiate_plot(dimx=24, dimy=9):
    plt.rcParams['figure.figsize'] = (dimx, dimy)
    global fig; fig = plt.figure()
    yield
    plt.show()

@contextmanager
def timeit():
    start = time.time()
    yield
    stop = time.time()
    print('This took %.2f seconds' % (stop-start))
    time.sleep(0.5)

In [2]:
%run '/home/ctherreau/initialize_midway.ipynb'

Initialization done, Notebook was last run on: 31/10/2018 at 08:00:53


In [3]:

import glob
def find_downloaded_events(df):
    global dsets
    dsets = hax.runs.datasets
    df['raw_data']=False
    for run_number in df.run_number.unique():
        run_name = dsets.loc[dsets.number==run_number, 'name'].values[0]
        for folder in glob.glob(hax.config['raw_data_local_path'][0]+'/*%s*'%run_name):
            for file in os.listdir(folder):
                min_event_number, max_event_number = file.split('-')[2:4]
                df.loc[(df.run_number==run_number) 
                       & (df.event_number>int(min_event_number))
                       & (df.event_number<int(max_event_number)),
                       'raw_data'
                      ] = True
    return df

In [8]:
############## Classifying Peaks #######################
import os
if not os.getcwd() in sys.path: sys.path.append(os.getcwd())
from MultipleS2Peaks import MultipleS2Peaks
ms2p = MultipleS2Peaks()

def tagging_peak_type(peak, s1, s2):
    if peak.type == 's1':
        return peak.type
    if peak.type == 's2':
        if peak.area<150:
            return 'single_e'
        drift_time = (peak.index_of_maximum - s1.index_of_maximum) * ms2p.sample_duration
        z = - ms2p.drift_velocity_liquid * (drift_time - ms2p.drift_time_gate)
        if z > 0: z=0
        for rp in peak.reconstructed_positions:
            if rp.algorithm == 'PosRecTopPatternFit':
                gof = getattr(rp, 'goodness_of_fit')
        ans = ms2p.determine_interaction(pd.DataFrame([dict(z=z, 
                                                     area=peak.area, 
                                                     goodness_of_fit_tpf=gof,
                                                     range_50p_area=list(peak.range_area_decile)[5],
                                                     s2=s2.area
                                                    )]))
        if not ans.not_interaction.values[0]:
            return 's2'  
        elif ans.not_interaction_pattern.values[0]:
            if (z > -0.1) and (drift_time > 0):
                return 's1_ap'
            return 'e_train'
        elif ans.not_interaction_width.values[0]:
            return 'long'
        elif ans.not_interaction_depth.values[0]: return 'outside'

In [9]:
############## Plotting #######################
def hit_color(hits):
    color_factor = np.clip(hits.height/hits.noise_sigma, 0, 15)/15
    is_rejected = hits.is_rejected.astype(int)

    aplist = c['DesaturatePulses.DesaturatePulses']['large_after_pulsing_channels']
    offlist = list(np.where(np.array(c['DEFAULT']['gains'])==0.0)[0])

    #is_ap = hits.channel.isin(aplist).astype(int)
    is_off = hits.channel.isin(offlist).astype(int)
    #is_any = 1- (1-is_ap) * (1-is_off) * (1-is_rejected)

    rgba_colors = np.zeros((len(hits), 4))
    rgba_colors[:, 0] = (1 - is_rejected) * color_factor
    rgba_colors[:, 1] = is_off
    rgba_colors[:, 2] = (1 - is_rejected) * (1 - color_factor)
    rgba_colors[:, 3] = 0.75
    
    return rgba_colors

def plot_2d_waveform(event, xlim, event2=None):

    if len(event.interactions) < 1:
        print('No interaction in this event')
        return 0

    s1 = event.peaks[event.interactions[0].s1]
    s2 = event.peaks[event.interactions[0].s2]
        
    with initiate_plot(30,15):
        axm = plt.gca()
        axm.tick_params(axis='both', bottom='off', labelbottom='off', left='off', labelleft='off')
        pos = axm.get_position()
        top = [pos.x0, pos.y0+0.5*pos.height, pos.width, pos.height*0.5]
        bot = [pos.x0, pos.y0, pos.width, pos.height*0.5]

        ####################################################
        axt = plt.axes(top)
        w = event.get_sum_waveform('tpc').samples[:]
        time = np.arange(len(w))*0.01

        ymax = np.max(w)**1.5

        plt.yscale('symlog')
        plt.plot(time, w, color='k', lw=2.0, zorder=1)
        
        if event2:
            w2 = event2.get_sum_waveform('tpc').samples[:]
            plt.plot(time, w2, color='b', ls='-', lw=2.0, zorder=0)

        plt_config(xlim = xlim, ylim=[-1, ymax], ylabel='Amplitude [pe/bin]')
        axt.tick_params(axis='x', bottom='off', labelbottom='off', left='off', labelleft='off')

        ####################################################
        axb = plt.axes(bot)
        plt.axhline(127, color='k', zorder=0)

        for peak_i, peak in enumerate(event.peaks):
            if peak.detector != 'tpc': continue
            if peak.right/100 < xlim[0] or peak.left/100 > xlim[1]: continue

            hits = pd.DataFrame(peak.hits)
            hits = hits[(hits.channel<248) & (hits.area>2)]            
            axb.scatter(hits.index_of_maximum*0.01, hits.channel, 
                        c=hit_color(hits), edgecolor='none', s=10 * np.clip(hits.area, 0, 10))

            if peak.type == 's1' or (peak.area>100 and peak.type == 's2'):
                x, y = peak.index_of_maximum*0.01, w[peak.index_of_maximum]
                ytext = np.random.uniform(y, min(ymax, y*5))

                if x>xlim[1] or x<xlim[0]:
                    continue
                    
                text = tagging_peak_type(peak, s1, s2)
                
                axt.axvspan(hits.index_of_maximum.min()*0.01, (hits.index_of_maximum.max()-1)*0.01, 
                    color='k', alpha=0.2, lw=3.0, fc='grey')
                axb.axvspan(hits.index_of_maximum.min()*0.01, (hits.index_of_maximum.max()-1)*0.01,
                    color='k', alpha=0.2, lw=3.0, fc='grey')
                axt.text(x, ytext, text)

                cmap = dict(s1='C0', s2='C1', e_train='C2' ,long='C3', single_e='C4', outside='C5', s1_ap='C6')
                axt.scatter(x, y, color=cmap[text])
        
        plt_config(xlim=xlim, ylim=[0, 249], xlabel='Time [$\mu s$]', ylabel='PMT channel')

In [10]:
# # reading a dataframe containing event you are interested in
# # Require run_number and event_number column

# # For understanding background runs
# run_name = '170419_0228'
# my_dataset = 8861
# d = hax.minitrees.load(run_name, ['Fundamentals', 'Corrections', 'Basics', 'Extended'])
# d = d[(d.cs2>10000) & (d.z_3d_nn<-10)]

# d = find_downloaded_events(d)
# d = d[d.raw_data]
# len(d.event_number.unique())

# Look at AP desaturation 

In [11]:
import numpy as np
from pax import plugin
from pax.dsputils import adc_to_pe
from pax.plugins.signal_processing.DesaturatePulses import DesaturatePulses as LocalDesaturatePulses

class AfterPulsingCorrections(LocalDesaturatePulses):
    """Replace the tail of after pulsing channel with scaled sum waveform of other channels
    """

    def transform_event(self, event):
        tpc_channels = np.array(self.config['channels_in_detector']['tpc'])

        for pulse_i, pulse in enumerate(event.pulses):
            # Consider only pulses in after_pulsing channel
            if pulse.length < 300 or pulse.channel not in self.config['large_after_pulsing_channels']:
                continue

            w = self.waveform_in_pe(pulse)
            if max(w) < 10: continue # Skip if the pulse is too small

            other_pulses = [p for i, p in enumerate(event.pulses)
                if p.left < pulse.right and p.right > pulse.left and
                p.channel in tpc_channels and
                p.channel not in self.config['large_after_pulsing_channels']]

            #Build the (gain-weighted) sum waveform of the non-saturated pulses
            min_left = min([p.left for p in other_pulses + [pulse]])
            max_right = max([p.right for p in other_pulses + [pulse]])
            sumw = np.zeros(max_right - min_left + 1)
            for p in other_pulses:
                offset = p.left - min_left
                sumw[offset:offset + len(p.raw_data)] += self.waveform_in_pe(p)

            # Crop it to include just the part that overlaps with this pulse
            offset = pulse.left - min_left
            sumw = sumw[offset:offset + len(pulse.raw_data)]

            # Find all the peaks in this pulse above prominence of 1 pe * (sum(sumw) / sum(w))
            convx = np.arange(100)/100-0.495 # A mean 0 integer list
            # find slope by linear regresion with +-50 samples
            slope = np.convolve(-convx, sumw, mode='same')/np.var(convx*200)
            peaks = np.where((slope <= 0) & (np.hstack((-10, slope[:-1])) > 0) & (sumw > sumw.sum()/w.sum()))[0]

            lefts, rights = [], []
            for peak_i, peak in enumerate(peaks):
                # Find the left and right extention of each peak by conditioning on slope
                start = rights[peak_i-1] if peak_i>0 else 0
                end = peaks[peak_i+1]-100 if peak_i<len(peaks)-1 else len(sumw)-1

                right = np.where((slope<0.1) & (np.arange(len(slope))<end))[0]
                right = right[-1] if len(right)>0 else end

                left = np.where((slope>=0.1) & (np.arange(len(slope))>start))[0]
                left = left[0] if len(left)>0 else start

                rights.append(right)
                lefts.append(left)

                reference_slice = slice(left, peak)
                ratio = w[reference_slice].sum()/sumw[reference_slice].sum()

                correction_slice = slice(peak, right)
                w[correction_slice] = np.clip(sumw[correction_slice] * ratio, 0, float('inf'))

            # Convert back to raw ADC counts and store the corrected waveform
            # Note this changes the type of pulse.w from int16 to float64: we don't have a choice,
            # int16 probably can't contain the large amplitudes we may be putting in.
            # As long as the raw data isn't saved again after applying this correction, this should be no problem
            # (as in later code converting to floats is anyway the first step).
            w /= adc_to_pe(self.config, pulse.channel)
            w = self.reference_baseline - w - pulse.baseline

            pulse.raw_data = w

        return event

In [12]:
# Process raw data from the start till finish building peaks
from pax import plugin, datastructure, dsputils
from pax.plugins.signal_processing import CheckPulses, PulseProperties, DesaturatePulses, HitFinder, BuildPeaks, SumWaveform
from pax.plugins.peak_processing import RejectNoiseHits, LocalMinimumClustering, NaturalBreaksClustering, BasicProperties, ClassifyPeaks
from pax.plugins.interaction_processing import BuildInteractions, RZCorrection, S1AreaFractionTopProbability

import pprint

class SignalProcessing(plugin.TransformPlugin):
    def transform_event(self, event, switch_out=False, xlim=[900, 1800]):
        self.config.update(self.config['pax'])
        self.config.update(self.config['DEFAULT'])
        plugin_list = self.config['dsp']+self.config['pre_analysis']
        
        #plugin_list.remove('BuildInteractions.BasicInteractionProperties')
        plugin_list.remove('HitfinderDiagnosticPlots.HitfinderDiagnosticPlots')
        
        switch = plugin_list.index('DesaturatePulses.DesaturatePulses')
        #plugin_list[switch] = 'DesaturatePulses'
        plugin_list.insert(switch+1, 'AfterPulsingCorrections')
        
        for ix, p in enumerate(plugin_list):
            #print(p)

            tmp = self.config
            for k in p.split('.')+[p]:
                if k in self.config.keys():
                    tmp.update(self.config[k])
                if 'AfterPulsingCorrections' == k:
                    tmp.update(self.config['DesaturatePulses.DesaturatePulses'])

            p = eval(p)(tmp, self.processor)
            event = p.transform_event(event)

        return event

In [13]:
# # Pax core processor can handle one folder (run) a time
# # Here we make a define smart iterator class
# # Switch to next run once exhaust all its events
# from pax import units
# from pax import configuration

# class WF_Events(object):
#     def __init__(self, df, max_iter=10):
#         self.df = df
#         self.df.sort_values(by=['run_number', 'event_number'], inplace=True)
#         self.run_number_iter = iter(df.run_number.unique())
#         self.a = 0
#         self.max_iter = max_iter

#     def init_pax_processor(self, event_numbers):
#         config_names=('_base', 'XENON1T')
#         global c
#         c = configuration.load_configuration(config_names)

#         from pax import core
#         self.core_processor = core.Processor(
#             config_names=('_base','XENON1T'),
#             just_testing=False,
#             config_dict={
#                 'pax':{
#                     'input_name' : '/project/lgrandi/xenon1t/raw_for_waveforms/%s'%self.run_name,
#                     'events_to_process' : event_numbers,
#                     'output' : 'Dummy.DummyOutput',
#                     'pre_output' : [],
#                     'encoder_plugin' : None,
#                     }}
#             )
#         events = self.core_processor.get_events()
#         return events
    
#     def init_my_processor(self, event_numbers):
#         self.sp = SignalProcessing(config_values=c.copy(), processor=self.core_processor)
#         events = hax.raw_data.raw_events(self.run_name, event_numbers)
#         return events

#     def __iter__(self):
#         self.run_number = next(self.run_number_iter)
#         tmp = self.df[self.df.run_number==self.run_number]
#         event_numbers = tmp.event_number.unique()
#         self.run_name = dsets.loc[dsets.number==self.run_number, 'name'].values[0]
#         print(self.run_name, len(event_numbers))

#         self.events = self.init_pax_processor(event_numbers)
#         self.my_events = self.init_my_processor(event_numbers)
#         return self

#     def __next__(self):
#         if self.a > self.max_iter:
#             raise StopIteration

#         try:
#             self.a += 1
#             return (next(self.events), self.sp.transform_event(next(self.my_events)))
#         except StopIteration:
#             try:
#                 self.__iter__()
#                 return self.__next__()
#             except StopIteration:
#                 print('Run out of event')
#                 raise StopIteration

In [15]:
d= hax.minitrees.load('Fax_HE_000000', 
                      ['Basics','Fundamentals'])

In [16]:
d.head()
run_name=[]
for i in range(0,len(d)):
     run_name.append('Fax_HE_000000')
d['run_name']=run_name

In [68]:
# Pax core processor can handle one folder (run) a time
# Here we make a define smart iterator class
# Switch to next run once exhaust all its events
from pax import units
from pax import configuration

class WF_Events_MC(object):
    def __init__(self, df, max_iter=10):
        self.df = df
        self.df.sort_values(by=['run_number', 'event_number'], inplace=True)
        self.run_number_iter = iter(df.run_number.unique())
        self.a = 0
        self.max_iter = max_iter

    def init_pax_processor(self, event_numbers):
        config_names=('_base', 'XENON1T','Simulation')
        global c
        c = configuration.load_configuration(config_names)

        from pax import core
        print(self.run_name)
        self.core_processor = core.Processor(
            config_names=('_base','XENON1T','Simulation'),
            just_testing=False,
            config_dict={
                'pax':{
                    'input_name' : '/dali/lgrandi/ctherreau/sim/Fax_HE_normal_pax/raw/%s'%self.run_name,
                    'events_to_process' : event_numbers,
                    'output' : 'Dummy.DummyOutput',
                    'pre_output' : [],
                    'encoder_plugin' : None,
                    }}
            )
        events = self.core_processor.get_events()
        return events
    
    def init_my_processor(self, event_numbers):
        self.sp = SignalProcessing(config_values=c.copy(), processor=self.core_processor)
        events = hax.raw_data.raw_events(self.run_name, event_numbers)
        return events

    def __iter__(self):
        self.run_number = next(self.run_number_iter)
        tmp = self.df[self.df.run_number==self.run_number]
        event_numbers = tmp.event_number.unique()
        print( event_numbers)
        
        self.run_name =self.df.run_name.values[0] #dsets.loc[dsets.number==self.run_number, 'name'].values[0]
        print(self.run_name, len(event_numbers))

        self.events = self.init_pax_processor(event_numbers)
        self.my_events = self.init_my_processor(event_numbers)
        return self

    def __next__(self):
        if self.a > self.max_iter:
            raise StopIteration

        try:
            self.a += 1
            return (next(self.events), self.sp.transform_event(next(self.my_events)))
        except StopIteration:
            try:
                self.__iter__()
                return self.__next__()
            except StopIteration:
                print('Run out of event')
                raise StopIteration

In [18]:
d = find_downloaded_events(d)
#d = d[d.raw_data]
#len(d.event_number.unique())

In [19]:
d

,drift_time,event_duration,event_number,event_time,largest_coincidence,largest_other_s1,largest_other_s2,largest_unknown,largest_veto,run_number,...,s1_area_fraction_top,s1_range_50p_area,s2,s2_area_fraction_top,s2_range_50p_area,x_pax,y_pax,z,run_name,raw_data
0,687050.0,720524,0,1540541697472662528,0,0.000000,7413.342285,6.632317,0,10000,...,0.066861,78.619527,7.798501e+05,0.631200,2143.611940,12.094611,13.590852,-91.494225,Fax_HE_000000,False
1,496110.0,529362,1,1540541716207344896,0,0.000000,4053.447510,0.000000,0,10000,...,0.136171,75.188323,4.936871e+05,0.632382,1844.235526,26.059525,1.371554,-66.003738,Fax_HE_000000,False
2,303630.0,336708,2,1540541729767129344,0,1.991732,3563.872314,0.000000,0,10000,...,0.213805,78.294936,4.783610e+05,0.628823,1476.728502,-33.540726,-6.857769,-40.307655,Fax_HE_000000,False
3,95340.0,127689,3,1540541747921881088,0,0.000000,14082.302734,2.020415,0,10000,...,0.355290,76.450193,1.098839e+06,0.634001,865.458290,19.077068,10.348997,-12.500940,Fax_HE_000000,False
4,505940.0,540360,4,1540541765959893504,0,0.000000,4851.713867,0.000000,0,10000,...,0.120787,78.086075,4.098547e+05,0.630725,1865.112260,-0.872807,-33.041981,-67.316040,Fax_HE_000000,False
5,555960.0,587900,5,1540541782456429568,0,0.000000,8062.044922,5.871555,0,10000,...,0.112882,77.942772,8.802812e+05,0.631652,1918.912509,17.331453,-1.870301,-73.993713,Fax_HE_000000,False
6,659240.0,691628,6,1540541802367333120,0,0.000000,4531.486328,15.567016,0,10000,...,0.074044,78.454602,6.045174e+05,0.629734,2114.293419,8.104636,-16.333960,-87.781593,Fax_HE_000000,False
7,86820.0,118584,7,1540541823679610880,0,0.000000,18975.736328,18.732872,0,10000,...,0.329752,75.178683,1.038255e+06,0.632323,826.480708,-8.354010,-0.872807,-11.363520,Fax_HE_000000,False
8,244910.0,277008,8,1540541856082003200,0,0.000000,37936.394531,13.773777,0,10000,...,0.246741,78.072269,1.788798e+06,0.637635,1326.716932,11.346491,-6.857769,-32.468536,Fax_HE_000000,False
9,506220.0,539548,9,1540541884766978560,0,0.000000,4232.521973,20.204748,0,10000,...,0.120511,78.931129,6.122606e+05,0.636086,1897.413023,-3.615915,20.573309,-67.353416,Fax_HE_000000,False


In [69]:
evts = WF_Events_MC(d, max_iter=3)


In [70]:
from IPython.display import clear_output
# Every 10 output ask jupyter to clear output
# To avoid over use mem use for display
clear_output_counter = 0

for event_pax_core, event_my in evts:
    
    event_pax_core = evts.core_processor.process_event(event_pax_core)
    sec = d[(d.event_number==event_pax_core.event_number) & (d.run_number==evts.run_number)]
    if len(sec) < 1: continue
    print('here',event_pax_core.event_number)

    s2ct = event_pax_core.peaks[event_pax_core.interactions[0].s2].center_time/1000
    plot_2d_waveform(event_pax_core, [s2ct-5, s2ct+20], event_my)

    #fig.savefig('/home/zhut/Beta_Beta/Calibration_MS/waveforms/examples/run%d_event%d.png'%(evts.run_number, event_pax_core.event_number), dpi=fig.dpi)

    clear_output_counter += 1
    if clear_output_counter >= 10:
        clear_output()
        clear_output_counter = 0


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74
 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99]
Fax_HE_000000 100
Fax_HE_000000


processor MainProcess L66 INFO This is PAX version 6.10.1, running with configuration for XENON1T.


IsADirectoryError: [Errno 21] Is a directory: '/dali/lgrandi/ctherreau/sim/Fax_HE_normal_pax/raw/Fax_HE_000000'